In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import statsmodels.api as sm
import sklearn

### Import data

In [31]:
# import data
from help_functions import get_energy_data

energydata = get_energy_data.get_data()

# select data since two years
nrows = len(energydata)-17472
energydata = energydata.iloc[nrows:]

100%|██████████| 255/255 [02:40<00:00,  1.59it/s]


### Create (aggregated) Dummies 

In [41]:
from help_functions import dummy_mapping

energydata = dummy_mapping.get_season_mapping(energydata)
energydata = dummy_mapping.get_day_mapping(energydata)
energydata = dummy_mapping.get_hour_mapping(energydata)

energydata

AttributeError: module 'help_functions.dummy_mapping' has no attribute 'get_hour_mapping'

In [34]:
from help_functions import dummy_mapping

energydata = (
    energydata
    .pipe(dummy_mapping.get_season_mapping())
    .pipe(dummy_mapping.get_day_mapping())
    .pipe(dummy_mapping.get_hour_mapping())
)

TypeError: 'DataFrame' object is not callable

In [15]:
# add weekdays, months, hours seperately
# monday = 0, sunday = 6
energydata['weekday'] = energydata.index.weekday
energydata['hour'] = energydata.index.hour
energydata['month'] = energydata.index.month


# create winter/cold dummy variable
energydata['winter'] = energydata['month'].apply(
    lambda x: 1 if x in [10, 11, 12, 1, 2, 3] else 0)

# Define mapping of hours to timeframes (based on graph) and create dummy variable
time_mapping = {
    'high_consumption_time': list(range(7, 20)),
    'transition_time': [6, 20, 21, 22, 23]}

for timeframe, hours in time_mapping.items():
    energydata[timeframe] = energydata['hour'].apply(
        lambda x: 1 if x in hours else 0)

# create weekend day dummy variable
energydata['weekend_day'] = energydata['weekday'].apply(
    lambda x: 1 if x in [5, 6] else 0)

# drop unneccesary columns
energydata = energydata.drop(
    columns=['weekday', 'hour', 'month'])

### Quantile Regression

In [16]:
y_ec = energydata['energy_consumption']
X_ec = energydata.drop(
    columns=['energy_consumption'])  

# add constant for the intercept term
X_ec = sm.add_constant(X_ec)

X_ec

,const,winter,high_consumption_time,transition_time,weekend_day
date_time,,,,,
2021-11-14 14:00:00,1.0,1,1,0,1
2021-11-14 15:00:00,1.0,1,1,0,1
2021-11-14 16:00:00,1.0,1,1,0,1
2021-11-14 17:00:00,1.0,1,1,0,1
2021-11-14 18:00:00,1.0,1,1,0,1
...,...,...,...,...,...
2023-11-12 09:00:00,1.0,1,1,0,1
2023-11-12 10:00:00,1.0,1,1,0,1
2023-11-12 11:00:00,1.0,1,1,0,1


In [26]:
from help_functions import get_forecast_timestamps

energyforecast = get_forecast_timestamps.forecast_timestamps(energydata.index[-1])

energyforecast['weekday'] = energyforecast.index.weekday
energyforecast['hour'] = energyforecast.index.hour
energyforecast['month'] = energyforecast.index.month

# create winter/cold dummy variable
energyforecast['winter'] = energyforecast['month'].apply(
    lambda x: 1 if x in [10, 11, 12, 1, 2, 3] else 0)

# time mapping already initialized
for timeframe, hours in time_mapping.items():
    energyforecast[timeframe] = energyforecast['hour'].apply(
        lambda x: 1 if x in hours else 0)

# create weekend day dummy variable
energyforecast['weekend_day'] = energyforecast['weekday'].apply(
    lambda x: 1 if x in [5, 6] else 0)

energyforecast = energyforecast.drop(columns={'hour', 'month', 'weekday'})

### Quantilregression

In [27]:
X_ec
X_fc
energyforecast

,winter,high_consumption_time,transition_time,weekend_day
date_time,,,,
2023-11-12 14:00:00,1,1,0,1
2023-11-12 15:00:00,1,1,0,1
2023-11-12 16:00:00,1,1,0,1
2023-11-12 17:00:00,1,1,0,1
2023-11-12 18:00:00,1,1,0,1
...,...,...,...,...
2023-11-17 09:00:00,1,1,0,0
2023-11-17 10:00:00,1,1,0,0
2023-11-17 11:00:00,1,1,0,0


In [28]:
X_fc = sm.add_constant(energyforecast, has_constant='add')

quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]

model_qr = sm.QuantReg(y_ec, X_ec)

for q in quantiles:

    model_temp = model_qr.fit(q=q)

    # Calculate forecasts for X_fc using the fitted model for the current quantile
    forecast_temp = model_temp.predict(X_fc)

    # Add the forecasts to the energy_forecast DataFrame with a label like 'forecast025'
    energyforecast[f'forecast{q}'] = forecast_temp

In [29]:
energyforecast

,winter,high_consumption_time,transition_time,weekend_day,forecast0.025,forecast0.25,forecast0.5,forecast0.75,forecast0.975
date_time,,,,,,,,,
2023-11-12 14:00:00,1,1,0,1,43.324500,51.660251,55.736549,60.040337,66.666000
2023-11-12 15:00:00,1,1,0,1,43.324500,51.660251,55.736549,60.040337,66.666000
2023-11-12 16:00:00,1,1,0,1,43.324500,51.660251,55.736549,60.040337,66.666000
2023-11-12 17:00:00,1,1,0,1,43.324500,51.660251,55.736549,60.040337,66.666000
2023-11-12 18:00:00,1,1,0,1,43.324500,51.660251,55.736549,60.040337,66.666000
...,...,...,...,...,...,...,...,...,...
2023-11-17 09:00:00,1,1,0,0,51.296627,61.772501,65.133511,68.803253,75.399502
2023-11-17 10:00:00,1,1,0,0,51.296627,61.772501,65.133511,68.803253,75.399502
2023-11-17 11:00:00,1,1,0,0,51.296627,61.772501,65.133511,68.803253,75.399502
